In [ ]:
#!pip install pandas_datareader
#!pip install numpy
#!pip install tensorflow
#!pip install websocket
#!pip install websocket-client

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [9]:
import plotly.graph_objs as go
import pandas as pd
import requests

# URL para obter dados históricos do preço do Bitcoin
url = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=2010-07-17&end=2024-03-26'

# Obter os dados JSON da URL
response = requests.get(url)
data = response.json()

# Extrair datas e preços do JSON
dates = list(data['bpi'].keys())
prices = list(data['bpi'].values())

# Converter as datas para o formato datetime
dates = pd.to_datetime(dates)

# Criar um DataFrame pandas com datas e preços
df = pd.DataFrame({'Date': dates, 'Price': prices})

# Criar um gráfico interativo com Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines', name='Preço do Bitcoin', line=dict(color='orange', width=2)))

fig.update_layout(title='Preço do Bitcoin desde sua criação até hoje',
                  xaxis_title='Data',
                  yaxis_title='Preço (USD)',
                  xaxis=dict(type='date'),
                  yaxis=dict(type='linear'),
                  showlegend=True,
                  plot_bgcolor='white')

# Adicionar controle deslizante para selecionar anos
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    width=1000,
    height=700 
)

fig.show()


# linear

In [8]:
import plotly.graph_objs as go
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression

# URL para obter dados históricos do preço do Bitcoin
url = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=2010-07-17&end=2024-03-26'

# Obter os dados JSON da URL
response = requests.get(url)
data = response.json()

# Extrair datas e preços do JSON
dates = list(data['bpi'].keys())
prices = list(data['bpi'].values())

# Converter as datas para o formato datetime
dates = pd.to_datetime(dates)

# Criar um DataFrame pandas com datas e preços
df = pd.DataFrame({'Date': dates, 'Price': prices})

# Adicionando características adicionais para o modelo de regressão linear
df['Days_since_start'] = (df['Date'] - df['Date'].min()).dt.days

# Treinando o modelo de regressão linear com todos os dados
X = df[['Days_since_start']].values
y = df['Price'].values
model = LinearRegression()
model.fit(X, y)

# Criar um array de datas para fazer previsões desde a criação do Bitcoin até hoje
all_dates = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
all_days_since_start = (all_dates - df['Date'].min()).days.values.reshape(-1, 1)

# Fazer previsões para todos os dias
predicted_prices = model.predict(all_days_since_start)

# Criar um gráfico interativo com Plotly
fig = go.Figure()

# Adicionar dados históricos
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines', name='Preço do Bitcoin (Histórico)', line=dict(color='orange', width=2)))

# Adicionar linha de previsão
fig.add_trace(go.Scatter(x=all_dates, y=predicted_prices, mode='lines', name='Previsão do Preço do Bitcoin', line=dict(color='blue', width=2)))

fig.update_layout(title='Preço do Bitcoin e Previsão',
                  xaxis_title='Data',
                  yaxis_title='Preço (USD)',
                  xaxis=dict(type='date'),
                  yaxis=dict(type='linear'),
                  showlegend=True,
                  plot_bgcolor='white')

# Adicionar controle deslizante para selecionar anos
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    width=1000,
    height=700 
)
fig.show()


# Testing

# best so far

In [10]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# URL para obter dados históricos do preço do Bitcoin
url = 'https://api.coindesk.com/v1/bpi/historical/close.json?start=2010-07-17&end=2024-03-26'

# Obter os dados JSON da URL
response = requests.get(url)
data = response.json()

# Extrair datas e preços do JSON
dates = list(data['bpi'].keys())
prices = list(data['bpi'].values())

# Converter as datas para o formato datetime
dates = pd.to_datetime(dates)

# Criar um DataFrame pandas com datas e preços
df = pd.DataFrame({'Date': dates, 'Price': prices})

# Copiar o DataFrame antes de normalizá-lo
df_original = df.copy()

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
df['Price'] = scaler.fit_transform(df['Price'].values.reshape(-1,1))

# Restaurar os dados originais do preço do Bitcoin
df['Original_Price'] = df_original['Price']

# Função para criar o dataset para treinamento
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Definir o número de time steps
TIME_STEPS = 30

# Dividir os dados em conjuntos de treinamento e teste
train_size = int(len(df) * 0.8)
train, test = df.iloc[0:train_size], df.iloc[train_size:]

# Criar o dataset de treinamento e teste
X_train, y_train = create_dataset(train[['Price']], train['Price'], TIME_STEPS)
X_test, y_test = create_dataset(test[['Price']], test['Price'], TIME_STEPS)

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Fazer previsões
y_pred = model.predict(X_test)

# Desnormalizar os dados
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))

# Calcular o erro quadrático médio
mse = mean_squared_error(y_test, y_pred)
print("Erro quadrático médio:", mse)

# Criar um gráfico interativo com Plotly para visualizar as previsões
fig = go.Figure()


fig.add_trace(go.Scatter(x=test.index[TIME_STEPS:], y=y_pred.flatten(), mode='lines', name='Previsões', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=test.index, y=test['Original_Price'], mode='lines', name='Preço do Bitcoin Original', line=dict(color='orange', width=2)))

fig.update_layout(title='Preço do Bitcoin e Previsões com LSTM',
                  xaxis_title='Data',
                  yaxis_title='Preço (USD)',
                  showlegend=True,
                  plot_bgcolor='white')

# Adicionar controle deslizante para selecionar anos
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    width=1000,
    height=700 
)

fig.show()

Epoch 1/10


C:\Users\charl\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



95/95 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 0.0098
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 5.8070e-04
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 5.8131e-04
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 4.6391e-04
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 4.2904e-04
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4.4524e-04
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4.8484e-04
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 3.2823e-04
Epoch 9/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 2.5335e-04
Epoch 10/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.4592e-04
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
Erro quadrático médio: 2447973.4655734063
